# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# checking your current working directory
print(os.getcwd())

/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb


In [4]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = glob.glob(os.path.join(os.getcwd(), 'event_data', '*.csv'))

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list.extend(glob.glob(os.path.join(root, '*.csv')))
    print(file_path_list)

['/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-01-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-02-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-03-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-04-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-05-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-06-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-07-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-08-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-09-events.csv', '/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb/event_data/2018-11-10-events.csv',

In [5]:
 #initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:5])  # يعرض أول 5 صفوف فقط

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


[['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8'], ['', 'Logged In', 'Kaylee', 'F', '2', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Upgrade', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ['Mr Oizo', 'Logged In', 'Kaylee', 'F', '3', 'Summers', '144.03873', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'Flat 55', '200', '1.54111E+12', '8']]


In [6]:
# check the number of rows in your csv file
with open('event_datafile_clean.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [7]:
import os
print(os.getcwd())


/mnt/c/Users/Huawei/Downloads/project-1b-project-template.ipynb


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

# Checking the connection to Docker
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

In [8]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("✅ Connected to Cassandra successfully!")
except Exception as e:
    print("❌ Connection failed:")
    print(e)


✅ Connected to Cassandra successfully!


## Begin writing  Apache Cassandra code in the cells below

#### Creating a Cluster

In [9]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# TO-DO: Create a Keyspace 
session.execute("""
CREATE KEYSPACE IF NOT EXISTS SPARKIFY
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [11]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')


#### Create queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


1.1 - Create songs table

In [12]:
session.execute("""
    CREATE TABLE IF NOT EXISTS song_info_by_session (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
""")


#### 1.2 - Insert records into `song_info_by_session` table

In [13]:
file = 'event_datafile_clean.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### 1.3 - Do a SELECT to verify that the data have been inserted the table

In [14]:
query = "SELECT artist, song, length FROM song_info_by_session WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


song_info_by_session


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### 2.1 - Create `users` table

In [71]:
#will use composite (user_id ,sessionId) as unique primary key
#add itemInSession as a clustering column to sort the data

query = "CREATE TABLE IF NOT EXISTS users"
query = query +"(user_id INT, \
                  sessionId INT, \
                  itemInSession INT, \
                  artist TEXT, \
                  song TEXT, \
                  first_name TEXT, \
                  last_name TEXT, \
                  PRIMARY KEY ((user_id, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [69]:
#session.execute("DROP TABLE IF EXISTS users")

#### checking the tables

In [15]:
rows = session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name = 'sparkify'")
for row in rows:
    print(row.table_name)


song_info_by_session
users


#### 2.2 - Insert records into `users` table

In [73]:
file = 'event_datafile_clean.csv'
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = " INSERT INTO users (user_id, sessionId, itemInSession, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))
    

#### 2.3 - Do a SELECT to verify that the data have been inserted into the table

In [16]:
query = "SELECT artist ,song , first_name , last_name FROM users WHERE user_id =10 AND sessionID =182"   
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [17]:
rr = session.execute("""SELECT * FROM song_info_by_session LIMIT 5""")
for r in rr :
    print("-",r)

- Row(sessionid=23, iteminsession=0, artist='Regina Spektor', length=191.08526611328125, song='The Calculation (Album Version)')
- Row(sessionid=23, iteminsession=1, artist='Octopus Project', length=250.95791625976562, song='All Of The Champs That Ever Lived')
- Row(sessionid=23, iteminsession=2, artist='Tegan And Sara', length=180.06158447265625, song='So Jealous')
- Row(sessionid=23, iteminsession=3, artist='Dragonette', length=153.39056396484375, song='Okay Dolores')
- Row(sessionid=23, iteminsession=4, artist='Lil Wayne / Eminem', length=229.58975219726562, song='Drop The World')


In [18]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
rows = session.execute("""
    SELECT artist, song, length 
    FROM song_info_by_session
    WHERE sessionId = 338 AND itemInSession = 4 """)

for row in rows:
    print(row)


                    

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [20]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_session_info (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        first_name text,
        last_name text,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
""")


In [25]:
rows = session.execute("""
    SELECT artist, song, first_name, last_name 
    FROM user_session_info 
    WHERE userId = 10 AND sessionId = 182
""")
for row in rows:
    print(row)


In [28]:
file = 'event_datafile_clean.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  
    
    for line in csvreader:
        query = "INSERT INTO user_session_info (userId, sessionId, itemInSession, artist, song, first_name, last_name)"
        query += " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

In [29]:
rows = session.execute("SELECT COUNT(*) FROM user_session_info")
for row in rows:
    print(row)


Row(count=6820)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [31]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = """SELECT artist, song, first_name, last_name 
           FROM user_session_info 
           WHERE userId = 10 AND sessionId = 182 
           ORDER BY itemInSession"""

rows = session.execute(query)
for row in rows:
    print(row)

                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [32]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
session.execute("""
    CREATE TABLE IF NOT EXISTS song_listeners_info (
        song text,
        userId int,
        first_name text,
        last_name text,
        PRIMARY KEY (song, userId)
    )
""")


                    

In [34]:
query = """SELECT first_name, last_name 
           FROM song_listeners_info 
           WHERE song = 'All Hands Against His Own'"""

rows = session.execute(query)
for row in rows:
    print(row)


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [ ]:
session.execute("DROP TABLE IF EXISTS user_session_info")
session.execute("DROP TABLE IF EXISTS song_listeners_info")


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()